In [11]:
%reload_ext autoreload
%autoreload 2

In [34]:
import hopsworks
import pandas as pd
import warnings # Mute warnings
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR
from src.data import fill_missing_datetimes, load_and_concatenate

In [2]:
warnings.filterwarnings("ignore")
print(hopsworks.__version__)

3.7.0


In [3]:
# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [26]:
cd extracted_data/

C:\Users\PC\Documents\Proyect\extracted_data\Passenger-Flow-Prediction-CABA\extracted_data


In [35]:
ts_combined = load_and_concatenate("2023_total_pax.csv", "2024_total_pax.csv")
ts_combined

,line,station,hour_of_entry,total_pax
0,LineaB,Leandro N. Alem,2023-01-01 08:00:00,34.0
1,LineaB,Leandro N. Alem,2023-01-01 09:00:00,34.0
2,LineaB,Leandro N. Alem,2023-01-01 10:00:00,59.0
3,LineaB,Leandro N. Alem,2023-01-01 11:00:00,41.0
4,LineaB,Leandro N. Alem,2023-01-01 12:00:00,67.0
...,...,...,...,...
401545,LineaC,Diagonal Norte,2024-05-31 19:00:00,153.0
401546,LineaC,Diagonal Norte,2024-05-31 20:00:00,199.0
401547,LineaC,Diagonal Norte,2024-05-31 21:00:00,127.0
401548,LineaC,Diagonal Norte,2024-05-31 22:00:00,83.0


In [37]:
ts_combined['station'] = ts_combined['station'].replace({
    'Saenz PeÃ±a ': 'Saenz Pena',
    'Saenz PeÃ\x83Â\x83Ã\x82Â±a ': 'Saenz Pena',
    'Saenz PeÃ\x83Â±a ': 'Saenz Pena',
    'Callao.B' : 'Callao'
})

print(ts_combined[ts_combined['line'] == 'LineaA'].station.unique())
print(ts_combined[ts_combined['line'] == 'LineaB'].station.unique())
print(ts_combined[ts_combined['line'] == 'LineaC'].station.unique())

['Acoyte' 'Carabobo' 'Congreso' 'Plaza de Mayo' 'Castro Barros'
 'Primera Junta' 'Rio de Janeiro' 'Loria' 'San Pedrito' 'Flores'
 'Plaza Miserere' 'Lima' 'Peru' 'Piedras' 'Alberti' 'Puan' 'Saenz Pena'
 'Pasco']
['Leandro N. Alem' 'Federico Lacroze' 'Pasteur' 'Dorrego' 'Malabia'
 'Medrano' 'Florida' 'Callao' 'Pueyrredon' 'Rosas' 'Carlos Pellegrini'
 'Uruguay' 'Carlos Gardel' 'Los Incas' 'Echeverria' 'Tronador'
 'Angel Gallardo']
['Constitucion' 'Retiro' 'Avenida de Mayo' 'Independencia'
 'Mariano Moreno' 'General San Martin' 'Lavalle' 'San Juan'
 'Diagonal Norte']


In [38]:
ts_filled = fill_missing_datetimes(ts_combined)

len(ts_filled)

544430

In [39]:
ts_filled.to_csv('2324_together_total_pax.csv')

In [55]:
ts_filled

,hour_of_entry,line,station,total_pax
0,2023-01-01 08:00:00,LineaA,Acoyte,22.0
1,2023-01-01 09:00:00,LineaA,Acoyte,23.0
2,2023-01-01 10:00:00,LineaA,Acoyte,40.0
3,2023-01-01 11:00:00,LineaA,Acoyte,17.0
4,2023-01-01 12:00:00,LineaA,Acoyte,83.0
...,...,...,...,...
544425,2024-05-31 19:00:00,LineaC,San Juan,199.0
544426,2024-05-31 20:00:00,LineaC,San Juan,208.0
544427,2024-05-31 21:00:00,LineaC,San Juan,161.0
544428,2024-05-31 22:00:00,LineaC,San Juan,253.0


In [41]:
HOPSWORKS_PROJECT_NAME = 'mlops_rm'

In [42]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/603286


In [43]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [52]:
FEATURE_GROUP_NAME = 'ts_stations_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [53]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency for each line and station",
    primary_key = ['line', 'hour_of_entry', 'station'],
    event_time='hour_of_entry',
)

In [54]:
feature_group.insert(ts_filled, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/603286/fs/599109/fg/1201489


Uploading Dataframe: 0.00% |          | Rows 0/544430 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: ts_stations_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/603286/jobs/named/ts_stations_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2cd8c7ebd50>, None)